# Data Preprocessing

In [1]:
import pandas as pd

In [25]:
sheet_names = [
    'Info',
    'Historical',
    'Income Statement',
    'Quarterly Income Statement',
    'Cashflow',
    'Institutional Holders',
    'Mutual Fund Holders',
    'Major Holders'
]
#riempire stocks di tutti i vari codici, fare la retrive di tutti i file e buttarli in df_stock per poi poter lavorare su tutti i dati
stocks = ['./data/MSFT.xlsx']
df_stock = pd.ExcelFile(stocks[0])


## Integrazione dei dati finanziarii
Colonne aggiunte:
- **Daily_Return**: rendimento giornaliero.
- **Target_1day**: indica se il prezzo di chiusura del giorno successivo sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_5days**: indica se il prezzo di chiusura a 5 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_30days**: indica se il prezzo di chiusura a 30 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.

Integrato i vari sheet  "Income Statement", "Quarterly Income Statement" e "Cashflow" in un singolo excel. NB: Dato che questi fogli contengono dati finanziari annuali o trimestrali un approccio comune è portare avanti l'ultimo valore noto per ogni giorno fino a quando non si dispone di un nuovo valore. Per alcunii anni finanziari sarà Nan perché non li abbiamo.

In [23]:
# TODO: da fare per ogni stock
# Loading the 'Historical' data stock
historical_data = df_stock.parse('Historical')

# Renaming and setting the Date column
historical_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
historical_data['Date'] = pd.to_datetime(historical_data['Date'])
historical_data.set_index('Date', inplace=True)

# Calculate daily return
historical_data['Daily_Return'] = historical_data['Close'].pct_change()

# Create target variables for next day, next 5 days and next 30 days
historical_data['Target_1day'] = (historical_data['Close'].shift(-1) > historical_data['Close']).astype(int)
historical_data['Target_5days'] = (historical_data['Close'].shift(-5) > historical_data['Close']).astype(int)
historical_data['Target_30days'] = (historical_data['Close'].shift(-30) > historical_data['Close']).astype(int)

# Drop rows with NaN values (will be present due to the shifting for target creation)
historical_data = historical_data.dropna()

# Display the updated dataframe
historical_data.head()
# Loading the 'Income Statement' data for XOM
income_statement = df_stock.parse('Income Statement')

# Transposing the data for easier integration
income_statement = income_statement.set_index('Unnamed: 0').transpose()
income_statement.index = pd.to_datetime(income_statement.index)

# Selecting some of the key financial metrics (you can add or remove based on relevance)
selected_metrics = [
    'Normalized EBITDA',
    'Total Unusual Items',
    'Total Unusual Items Excluding Goodwill'
]

income_statement = income_statement[selected_metrics]

# Merging the income statement data with the historical data
merged_data = historical_data.join(income_statement, how='left')

# Forward filling the NaN values
merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')

# Display the updated dataframe with integrated financial metrics
merged_data.head()
# Loading the 'Quarterly Income Statement' data for XOM
quarterly_income_statement = df_stock.parse('Quarterly Income Statement')

# Transposing the data for easier integration
quarterly_income_statement = quarterly_income_statement.set_index('Unnamed: 0').transpose()
quarterly_income_statement.index = pd.to_datetime(quarterly_income_statement.index)

# Selecting some of the key financial metrics (you can add or remove based on relevance)
selected_metrics_quarterly = [
    'Normalized EBITDA',
    'Total Unusual Items',
    'Total Unusual Items Excluding Goodwill'
]

quarterly_income_statement = quarterly_income_statement[selected_metrics_quarterly]

# Merging the quarterly income statement data with the existing dataframe
merged_data = merged_data.join(quarterly_income_statement, how='left', rsuffix='_quarterly')

# Forward filling the NaN values
merged_data[selected_metrics_quarterly] = merged_data[selected_metrics_quarterly].fillna(method='ffill')

# Display the updated dataframe with integrated quarterly financial metrics
merged_data.head()
# Loading the 'Cashflow' data for XOM
cashflow = df_stock.parse('Cashflow')

# Transposing the data for easier integration
cashflow = cashflow.set_index('Unnamed: 0').transpose()
cashflow.index = pd.to_datetime(cashflow.index)

# Selecting some of the key cashflow metrics (you can add or remove based on relevance)
selected_cashflow_metrics = [
    'Operating Cash Flow',
    'Capital Expenditure',
    'Free Cash Flow'
]

cashflow = cashflow[selected_cashflow_metrics]

# Merging the cashflow data with the existing dataframe
merged_data = merged_data.join(cashflow, how='left', rsuffix='_cashflow')

# Forward filling the NaN values
merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')

# Display the updated dataframe with integrated cashflow metrics
merged_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Return,Target_1day,Target_5days,Target_30days,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Normalized EBITDA_quarterly,Total Unusual Items_quarterly,Total Unusual Items Excluding Goodwill_quarterly,Operating Cash Flow,Capital Expenditure,Free Cash Flow
Date,,,,,,,,,,,,,,,,,,,,
1986-03-14,0.060396,0.063632,0.060396,0.062553,308160000,0.0,0.0,0.035712,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-17,0.062553,0.064172,0.062553,0.063632,133171200,0.0,0.0,0.017250,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-18,0.063632,0.064172,0.061475,0.062014,67766400,0.0,0.0,-0.025432,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-19,0.062014,0.062553,0.060396,0.060936,47894400,0.0,0.0,-0.017390,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-03-20,0.060936,0.060936,0.058779,0.059318,58435200,0.0,0.0,-0.026547,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Engineering
- **Medie mobili**: Calcoliamo le medie mobili a breve e lungo termine per il prezzo di chiusura, che sono comuni nel trading algoritmico. Ad esempio, medie mobili a 5, 10, 30 e 50 giorni.
- **RSI (Relative Strength Index)**: Questo è un indicatore di momentum che può aiutare a identificare se un'azione è in condizione di "overbought" o "oversold".
- **MACD (Moving Average Convergence Divergence)**: Un altro indicatore di momentum.
- **Bollinger Bands**: Questi sono basati su medie mobili e possono aiutare a identificare se un prezzo è relativamente alto o basso.
- **Volatilità**: Potremmo calcolare la volatilità come la deviazione standard dei rendimenti giornalieri in una finestra temporale specifica.

In [24]:
# TODO: da fare per ogni stock
# Moving Averages
merged_data['MA_5'] = merged_data['Close'].rolling(window=5).mean()
merged_data['MA_10'] = merged_data['Close'].rolling(window=10).mean()
merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
merged_data['MA_50'] = merged_data['Close'].rolling(window=50).mean()

# RSI
delta = merged_data['Close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
merged_data['RSI'] = 100 - (100 / (1 + rs))

# MACD
merged_data['MACD'] = merged_data['Close'].ewm(span=12, adjust=False).mean() - merged_data['Close'].ewm(span=26, adjust=False).mean()
merged_data['Signal_Line'] = merged_data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
merged_data['Bollinger_Mid_Band'] = merged_data['Close'].rolling(window=20).mean()
merged_data['Bollinger_Upper_Band']  = merged_data['Bollinger_Mid_Band'] + 1.96*merged_data['Close'].rolling(window=20).std()
merged_data['Bollinger_Lower_Band']  = merged_data['Bollinger_Mid_Band'] - 1.96*merged_data['Close'].rolling(window=20).std()

# Volatility
merged_data['Volatility'] = merged_data['Daily_Return'].rolling(window=5).std()

# Drop NaN values resulting from rolling calculations
merged_data = merged_data.dropna()

# Display the dataset with new features
merged_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Return,Target_1day,Target_5days,...,MA_10,MA_30,MA_50,RSI,MACD,Signal_Line,Bollinger_Mid_Band,Bollinger_Upper_Band,Bollinger_Lower_Band,Volatility
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,236.009950,238.238421,230.503152,230.671524,35694800,0.0,0.0,-0.019368,1,1,...,236.728018,252.713585,262.290984,14.540885,-8.486472,-8.007111,244.711395,263.946825,225.475964,0.015161
2023-03-31,282.512980,288.029195,281.786100,287.063354,32766000,0.0,0.0,0.014962,0,1,...,276.907108,262.233321,259.083778,77.530457,7.547206,6.135221,267.966125,291.834903,244.097346,0.014477
2023-06-30,337.036412,342.005902,336.487586,339.820526,26823800,0.0,0.0,0.016386,0,0,...,335.617419,330.415437,318.747694,60.315063,4.795936,5.838509,334.058224,345.465791,322.650658,0.015243


In [22]:
# da fare per ogni stock
output_filepath = "processed_nomedellostock.xlsx"
len(merged_data)
merged_data.to_excel(output_filepath)
